In [2]:
import httpx

import csv
import json
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np


In [3]:
import csv
import json
import os
import sys
from pathlib import Path
import pandas as pd

In [4]:
class FetchException(Exception):
    """
    Turn a httpx.Response into an exception.
    """

    def __init__(self, response: httpx.Response):
        super().__init__(
            f"{response.status_code} retrieving {response.url}: {response.text}"
        )

In [5]:

def combine_url_with_params(url, params):
    """
    Use httpx.URL to create a URL joined to its parameters, suitable for use
    for cache keys.

    Parameters:
        - url: a URL with or without parameters already
        - params: a dictionary of parameters to add

    Returns:
        The URL with parameters added, for example:

        >>> combine_url_with_params(
            "https://example.com/api/",
            {"api_key": "abc", "page": 2}
        )
        "https://example.com/api/?api_key=abc&page=2"
    """
    url = httpx.URL(url)
    merged_params  = dict(url.params) | params  # merge the dictionaries
    encoded_url = str(url.copy_with(params=merged_params))
    return encoded_url.replace("%2C", ",")

In [6]:
combine_url_with_params(url, params)

NameError: name 'url' is not defined

In [ ]:

def cached_get(url) -> dict:
    """
    This function caches all GET requests it makes by writing
    the successful responses to disk.
    """
    full_url = combine_url_with_params(url, params)  # Ensure 'params' is defined in your context.
    
    try:
        response = httpx.get(full_url, follow_redirects=False, timeout=10000.0)
    except httpx.ReadTimeout as exc:
        # Log the error or handle it as needed.
        raise FetchException(f"Request to {full_url} timed out") from exc

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        raise FetchException(response)


In [ ]:

START_URL = "https://api.census.gov/data/2021/acs/acs1/pums"
CSV_COLUMNS = ("STATE","SEX","PUMA","RACBLK","AGEP","HISP","FHISP","PWGTP","SCH","ESR","HINCP","ADJINC","SCHG","SCHL")



def build_census_csv(output_filename: Path):
    """
    Create a CSV file populated with the following columns:

    - `state` - State
    - `sex` - sex of the informant.
    - `PUMA` - Earliest date in hearing.
    - `RACBLK` - URL to the plain text hearing transcript.
    - `AGEP` - URL of the API call that returned this record.
    - `HISP` - URL of the API call that returned this record.
    - `FHISP` - URL of the API call that returned this record.
    - `PWGTP` - URL of the API call that returned this record.
    - `SCH` - URL of the API call that returned this record.
    - `ESR` - URL of the API call that returned this record.
    - `HINCP` - 
    - `ADJINC` -
    - `SCHG` -
    - `SCHL` -

    Parameters:
        output_filename: Path object representing location to write file.
    """  
    results = []
    new_url = START_URL


    data = cached_get(new_url)
 # Convert to DataFrame if data is available
    if data:
        df = pd.DataFrame(data[1:], columns=data[0])  # First row is headers
        print(df.head())  # Print the first few rows for a quick look

        # Write the DataFrame to a CSV file
        df.to_csv(output_filename, index=False)
        print(f"CSV file created at: {output_filename}")
    else:
        print("No data available")

if __name__ == "__main__":
    build_census_csv("census_2023.csv")


  SEX   PUMA RACBLK AGEP HISP FHISP PWGTP SCH ESR   HINCP    ADJINC SCHG SCHL
0   1  00800      0   36   01     0    35   1   6  -60000  1.029928    0   11
1   1  01002      0   19   01     0    35   3   6  -60000  1.029928   15   19
2   1  01802      0   57   01     0    70   1   6  -60000  1.029928    0   22
3   1  00600      0   29   01     0   149   1   6  -60000  1.029928    0   14
4   1  03741      0   26   01     0   177   1   6  -60000  1.029928    0    1
CSV file created at: census_2021.csv


## Indicators

In [ ]:
def comparative_inidcators_data(csv_file):
    """
    load the census information and calculate National education markers to compare with 
    Chicago's
    """
    df = pd.read_csv(csv_file)

    df["SCHL"]  = pd.to_numeric(df["SCHL"])    #
    df["SCHG"]  = pd.to_numeric(df["SCHG"])    #
    df_clean_clean_clean_clean_clean["AGEP"]  = pd.to_numeric(df["AGEP"])    # Convert PUMA to numeric if it's not already
    df["PUMA"]  = pd.to_numeric(df["PUMA"])    # Convert weigths to numeric if it's not already
    df["PWGTP"] = pd.to_numeric(df["PWGTP"]) 
    
     # create chicago 
    df.loc[(df["PUMA"] >= 3151) & (df["PUMA"] <= 3168), "STATE"] = "CHICAGO"   

    return df



In [7]:
data_2023 = comparative_inidcators_data("census_2023.csv")
data_2023.head

NameError: name 'comparative_inidcators_data' is not defined

In [8]:
def chicago_df(csv_file):

    df = pd.read_csv(csv_file)
    chicago_df = df[(df["PUMA"] >= 3151) & (df["PUMA"] <= 3168)] # filter chicago
    return chicago_df


In [ ]:
def chicago_dataframe(csv_file, output_file):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)
    
    # Filter the DataFrame for rows where PUMA is between 3151 and 3168
    chicago_df = df[(df["PUMA"] >= 3151) & (df["PUMA"] <= 3168)]
    
    # Save the filtered DataFrame to the output file without the index column
    chicago_df.to_csv(output_file, index=False)
    


In [ ]:
chicago_dataframe("census_2023.csv", "chicago_20232")

In [ ]:
def cleaning_data(csv_file):
    """
    load a cvs file and clean:
    - transform variables from string to numeric 
    """
    df = pd.read_csv(csv_file)
    # convert to numeric
    cols = ["SCHL", "SCHG", "AGEP", "PUMA", "PWGTP", "HINCP"]
    df[cols] = df[cols].apply(pd.to_numeric)
    # we are assuming missing values are at random and droping them 
    df.loc[(df["HINCP"] == -60000), "HINCP"] = None
    df_clean = df.dropna(subset=['HINCP']) 

    return df_clean

In [ ]:
df = pd.read_csv("chicago_2023")

In [ ]:
cols = ["SCHL", "SCHG", "AGEP", "PUMA", "PWGTP", "HINCP"]
df[cols] = df[cols].apply(pd.to_numeric)

In [ ]:
df["SCHL"] = pd.to_numeric(df["SCHL"])
#
df["SCHG"] = pd.to_numeric(df["SCHG"])
#
df["AGEP"] = pd.to_numeric(df["AGEP"])
# Convert PUMA to numeric if it's not already
df["PUMA"] = pd.to_numeric(df["PUMA"])
# Convert weigths to numeric if it's not already
df["PWGTP"] = pd.to_numeric(df["PWGTP"])

In [ ]:
df.loc[(df["HINCP"] == -60000), "HINCP"] = None
df_clean = df.dropna(subset=['HINCP'])
df_clean

,STATE,SEX,PUMA,RACBLK,AGEP,HISP,FHISP,PWGTP,SCH,ESR,HINCP,ADJINC,SCHG,SCHL
876,17,1,3163,0,29,2,0,117,1,1,70000.0,1.019518,0,20
877,17,1,3156,0,79,1,0,51,1,1,491000.0,1.019518,0,15
878,17,2,3156,0,75,1,0,57,1,6,491000.0,1.019518,0,15
879,17,2,3157,1,53,1,0,249,1,1,78600.0,1.019518,0,19
880,17,2,3153,0,33,1,0,117,1,1,75000.0,1.019518,0,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18239,17,2,3153,0,59,1,0,122,1,3,74500.0,1.019518,0,19
18240,17,2,3153,0,20,1,0,88,3,1,74500.0,1.019518,15,19
18241,17,2,3166,1,78,1,0,42,1,6,58000.0,1.019518,0,22
18242,17,2,3155,0,28,1,1,300,3,1,98000.0,1.019518,15,22


# Education vars

In [15]:
def chicago_dataframe(csv_file, output_file):
    """
    Filters the PUMAS to only the Chicago city
    """
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)   
    # Filter the DataFrame for rows where PUMA is between 3151 and 3168
    chicago_df = df[(df["PUMA"] >= 3151) & (df["PUMA"] <= 3168)]
    # Save the filtered DataFrame to the output file without the index column
    chicago_df.to_csv(output_file, index=False)
    return chicago_df

,STATE,SEX,PUMA,RACBLK,AGEP,HISP,FHISP,PWGTP,SCH,ESR,HINCP,ADJINC,SCHG,SCHL
390,17,1,3168,1,54,1,0,32,1,6,-60000,1.019518,0,16
585,17,2,3160,1,23,1,0,82,3,1,-60000,1.019518,15,19
606,17,1,3163,1,41,1,0,68,1,6,-60000,1.019518,0,16
619,17,1,3161,1,66,1,0,29,1,6,-60000,1.019518,0,17
867,17,1,3151,1,28,1,0,13,1,6,-60000,1.019518,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3404057,17,2,3153,0,59,1,0,122,1,3,74500,1.019518,0,19
3404058,17,2,3153,0,20,1,0,88,3,1,74500,1.019518,15,19
3404648,17,2,3166,1,78,1,0,42,1,6,58000,1.019518,0,22
3404904,17,2,3155,0,28,1,1,300,3,1,98000,1.019518,15,22


In [14]:
def cleaning_data(csv_file):
    """
    load a cvs file and clean:
    - transform variables from string to numeric 
    - drop observations with no income information
    """
    df = pd.read_csv(csv_file)
    # convert to numeric
    cols = ["SCHL", "SCHG", "AGEP", "PUMA", "PWGTP", "HINCP"]
    df[cols] = df[cols].apply(pd.to_numeric)
    # we are assuming missing values are at random and droping them 
    df.loc[(df["HINCP"] == -60000), "HINCP"] = None
    df_clean = df.dropna(subset=['HINCP']) 

    return df_clean

In [10]:
def education_vars(csv_file):
    """ creates the education variables to be used
    in the analysis  
    """
    df = cleaning_data(csv_file)
    # years of education
    df.loc[df["SCHL"] <= 3, "years_education"] = 0
    df.loc[df["SCHL"] == 4, "years_education"] = 1
    df.loc[df["SCHL"] == 5, "years_education"] = 2
    df.loc[df["SCHL"] == 6, "years_education"] = 3
    df.loc[df["SCHL"] == 7, "years_education"] = 4
    df.loc[df["SCHL"] == 8, "years_education"] = 5
    df.loc[df["SCHL"] == 9, "years_education"] = 6
    df.loc[df["SCHL"] == 10, "years_education"] = 7
    df.loc[df["SCHL"] == 11, "years_education"] = 8
    df.loc[df["SCHL"] == 12, "years_education"] = 9
    df.loc[df["SCHL"] == 13, "years_education"] = 10
    df.loc[df["SCHL"] == 14, "years_education"] = 11
    df.loc[df["SCHL"] == 15, "years_education"] = 12
    df.loc[df["SCHL"] >= 16, "years_education"] = 13

 # School age population
    df["school_age"]   = ((df["AGEP"] >= 5) & (df["AGEP"] <=18)).astype(int)
 # School levels theoretical age 
    df["elementary"]    = ((df["AGEP"] >= 5)  & (df["AGEP"] <=10)).astype(int)
    df["middle"]        = ((df["AGEP"] >= 11) & (df["AGEP"] <=13)).astype(int)
    df["high_school"]   = ((df["AGEP"] >= 14) & (df["AGEP"] <=18)).astype(int)

    # School attendance SCH
    #  "0": "N/A (less than 3 years old)",
    #  "3": "Yes, private school or college or home school",
    #  "1": "No, has not attended in the last 3 months",
    #  "2": "Yes, public school or public college"

    # Compute binary school attendance
    df["school_attendance"] = (((df["SCH"] != "1") & (df["school_age"] == 1)).astype(int))

    # Create a mask for school-age observations
    mask = df["school_age"] == 1

    # Compute school attendance by education level
    df["atten_elementary"] = np.where(mask & (df["years_education"] <= 4) & (df["elementary"] ==1), 1, 0)
    df["atten_middle"]     = np.where(mask & df["years_education"].between(5, 7) & (df["middle"] ==1), 1, 0)
    df["atten_high_school"] = np.where(mask & df["years_education"].between(8, 11) & (df["high_school"] ==1), 1, 0)

    # List of columns that need to be weighted by PWGTP
    cols_to_weight = [
        "school_attendance", "school_age",
        "atten_elementary", "atten_middle", "atten_high_school",
        "elementary", "middle", "high_school"
    ]

    # Multiply each column by PWGTP and save as a new column with _w appended
    df[[f"{col}_w" for col in cols_to_weight]] = df[cols_to_weight].multiply(df["PWGTP"], axis=0)
    
    selected_columns_df = df[['PUMA', "atten_elementary_w", 'elementary_w', 
                          'atten_middle_w', 'middle_w', 
                          'atten_high_school_w', 'high_school_w']].groupby("PUMA").sum().reset_index()
    return selected_columns_df

In [11]:
def education_indicators(csv_file, year):
    """
    Creates education indicators, adds a 'year' column, and returns the aggregated DataFrame.
    """
    df = education_vars(csv_file)
    
    # Calculate attendance rates (as percentages)
    df["attendance_rate_elementary"] = (df["atten_elementary_w"] / df["elementary_w"]) * 100
    df["attendance_rate_middle"] = (df["atten_middle_w"] / df["middle_w"]) * 100
    df["attendance_rate_high"] = (df["atten_high_school_w"] / df["high_school_w"]) * 100
    
    # Add a new column for the year
    df["year"] = year
    return df

In [12]:
def process_multiple_years(file_year_pairs, output_file="census_df.csv"):
    """
    Processes multiple CSV files (one per year), appends the results with a 'year' column,
    and saves the final DataFrame to CSV.
    
    Parameters:
        file_year_pairs: list of tuples (csv_file, year)
        output_file: path for the output CSV file
    """
    dfs = []
    for csv_file, year in file_year_pairs:
        df = education_indicators(csv_file, year)
        dfs.append(df)
        
    final_df = pd.concat(dfs, ignore_index=True)
    final_df.to_csv(output_file, index=False)
    return final_df

In [18]:
if __name__ == "__main__":

    # corresponding to years 2021, 2022, and 2023.
    file_year_pairs = [
        ("chicago_census_2023.csv", 2021),
        ("chicago_census_2023.csv", 2022),
        ("chicago_census_2023.csv", 2023),
    ]
    final_df = process_multiple_years(file_year_pairs)

In [ ]:
 
attendance_rate_high_school = (df["atten_elementary_w"]/df["elementary_w"]) * 100
attendance_rate_middle = (df["atten_elementary_w"]/df["elementary_w"]) * 100

In [ ]:
selected_columns_df = df[['PUMA', "atten_elementary_w", 'elementary_w', 
                          'atten_middle_w', 'middle_w', 
                          'atten_high_school_w', 'high_school_w']].groupby("PUMA").sum().reset_index()


In [9]:
selected_columns_df["attendance_rate_elementary"] = (
    selected_columns_df["atten_elementary_w"] / selected_columns_df["elementary_w"]
)*100
selected_columns_df["attendance_rate_middle"] = (
    selected_columns_df["atten_middle_w"] / selected_columns_df["middle_w"]
)*100
selected_columns_df["attendance_rate_high"] = (
    selected_columns_df["atten_high_school_w"] / selected_columns_df["high_school_w"]
)*100

NameError: name 'selected_columns_df' is not defined

In [ ]:
selected_columns_df

NameError: name 'selected_columns_df' is not defined

In [ ]:
census_df = selected_columns_df.to_csv("census_df.csv", index=False)
census_df

In [19]:
pip install keplergl

     ---------------------------------------- 0.0/18.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/18.4 MB ? eta -:--:--
     ---------------------------------------- 0.2/18.4 MB 1.3 MB/s eta 0:00:15
      --------------------------------------- 0.3/18.4 MB 2.1 MB/s eta 0:00:09
     - -------------------------------------- 0.5/18.4 MB 2.5 MB/s eta 0:00:08
     - -------------------------------------- 0.7/18.4 MB 2.9 MB/s eta 0:00:06
     -- ------------------------------------- 1.1/18.4 MB 3.6 MB/s eta 0:00:05
     --- ------------------------------------ 1.5/18.4 MB 4.2 MB/s eta 0:00:05
     ---- ----------------------------------- 1.9/18.4 MB 4.6 MB/s eta 0:00:04
     ----- ---------------------------------- 2.3/18.4 MB 5.3 MB/s eta 0:00:04
     ------ --------------------------------- 2.8/18.4 MB 5.6 MB/s eta 0:00:03
     ------- -------------------------------- 3.3/18.4 MB 6.1 MB/s eta 0:00:03
     -------- ------------------------------- 3.7/18.4 MB 6.3 MB/s


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\lopez\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
